In [5]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
con = duckdb.connect("../satan.db")

In [16]:
print(duckdb.__version__)
benchmark = con.sql("select id from benchmarks order by id desc limit 1").fetchall()

if len(benchmark) != 1:
    print("No benchmark found")
    
    raise Error("No analysis possible")
else:
    benchmark = benchmark[0][0]

con.sql("""select
        solvers.name, satisfiable, count(runs.id)
        from runs join solvers on (runs.solver = solvers.id)
        where benchmark = (select id from benchmarks order by id desc limit 1)
        group by satisfiable, solvers.name
        order by solvers.name;
        """)

0.7.1
1


┌─────────┬─────────────┬────────────────┐
│  name   │ satisfiable │ count(runs.id) │
│ varchar │    int8     │     int64      │
├─────────┼─────────────┼────────────────┤
│ cadical │           0 │           1894 │
│ cadical │           1 │           4406 │
│ cadical │          -1 │           1504 │
│ minisat │           0 │           2468 │
│ minisat │           1 │           3673 │
│ minisat │          -1 │           1824 │
└─────────┴─────────────┴────────────────┘